<a href="https://colab.research.google.com/github/jnises/llmog/blob/finetuning/training/quantize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# quantize the model and upload to huggingface
# uses unsloth for the quantization, so needs a gpu.
# it is probably much better to do the quantization some other way.

In [ ]:
# TODO specify the exact versions of dependencies here. This was written around the end of May 2025, so if unsloth has breaking changes try reverting to the version that was current at that time
import os
is_colab = any('COLAB_' in k for k in os.environ.keys())
if is_colab:
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install transformers==4.51.3
    !pip install --no-deps unsloth

In [ ]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')
model_name = 'gemma-3-1b-llmog'
quantized_model_name = 'gemma-3-1b-llmog-GGUF'
HF_REPO = f"jnises/{model_name}"
QUANTIZED_HF_REPO = f"jnises/{quantized_model_name}"

In [ ]:
from unsloth import FastModel
model, tokenizer = FastModel.from_pretrained(
    model_name = HF_REPO,
)

In [ ]:
model.save_pretrained_merged(quantized_model_name, tokenizer)

In [ ]:
model.save_pretrained_gguf(
        # need to be just the model name here, a full path results in confusing error messages
        quantized_model_name,
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

In [ ]:
api.upload_file(path_or_fileobj='gemma-3-1b-llmog-GGUF.Q8_0.gguf', path_in_repo='gemma-3-1b-llmog-GGUF.Q8_0.gguf', repo_id=QUANTIZED_HF_REPO, repo_type='model', token=HF_TOKEN)